In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
confusion_m = np.array([[118., 12.], [ 47., 15.]])

In [113]:
def evaluation(mat):
    acc = (mat[0][0] + mat[1][1])/mat.sum()
    err = 1 - acc
    recall = mat[1][1]/(mat[1][1]+mat[1][0])
    perci = mat[1][1]/(mat[1][1]+mat[0][1])
    spec = mat[0][0]/(mat[0][0]+mat[0][1])
    f1 = 2* (perci*recall)/(perci+recall)
    return acc, perci, recall, spec, f1

In [80]:
print(elavuation(confusion_m))

(0.6927083333333334, 0.30729166666666663, 0.24193548387096775, 0.5555555555555556, 0.9076923076923077, 0.3370786516853933)


In [81]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

pima = pd.read_csv('diabetes.csv')
print(pima.columns)
print(pima.head())

feature_cols = ['Pregnancies', 'Insulin', 'BMI', 'Age']

X = pima[feature_cols]
# print(X)
# y is a vector, hence we use dot to access 'label'
y = pima['Outcome']

# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


logreg = LogisticRegression()

# fit model
logreg.fit(X_train, y_train)

y_pred_prob = logreg.predict_proba(X_test)
y_pred = logreg.predict(X_test)

Index([u'Pregnancies', u'Glucose', u'BloodPressure', u'SkinThickness',
       u'Insulin', u'BMI', u'DiabetesPedigreeFunction', u'Age', u'Outcome'],
      dtype='object')
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [97]:
def proba_to_pred(yproba, ratio):
    return [0 if yproba[i][1] < ratio else 1 for i in range(len(yproba))]

In [98]:
def same(ls, ls2):
    for i in range(len(ls)):
        if ls[i] != ls2[i]:
            return False
    return True

In [99]:
def confusion(yt, yp):
    ma = np.zeros((2,2))
    for i in range(len(yt)):
        if yt[i] == yp[i] and yp[i] == 0:
            ma[0][0] += 1
        elif yt[i] == yp[i] and yp[i] == 1:
            ma[1][1] += 1
        elif yt[i] != yp[i] and yp[i] == 0:
            ma[1][0] += 1
        elif yt[i] != yp[i] and yp[i] == 1:
            ma[0][1] += 1
    return ma

In [100]:
float(y_train.value_counts()[1])/len(y_train)

0.3576388888888889

In [101]:
my_predict = proba_to_pred(y_pred_prob, 0.3576388888888889)

In [102]:
same(my_predict, y_pred)

False

In [103]:
confusion(list(y_test), my_predict)

array([[87., 43.],
       [24., 38.]])

In [110]:
from sklearn.preprocessing import binarize
y_pred_prob_class_1 = logreg.predict_proba(X_test)[:,1]
y_pred_threshold = binarize(y_pred_prob_class_1.reshape(1,-1), 0.335195)[0]

In [111]:
y_pred_threshold

array([1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1.,
       0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1.,
       0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0.,
       1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0.,
       1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 0.])

In [114]:
evaluation(confusion(list(y_test), y_pred_threshold))

(0.671875,
 0.4945054945054945,
 0.7258064516129032,
 0.6461538461538462,
 0.5882352941176471)

In [115]:

from sklearn.model_selection import cross_val_score 
import pandas as pd
from sklearn.linear_model import LogisticRegression

pima = pd.read_csv('diabetes.csv')

feature_cols = ['Pregnancies', 'Insulin', 'BMI', 'Age']

# X is a matrix,access the features we want in feature_cols
X = pima[feature_cols]

# y is a vector, hence we use dot to access 'label'
y = pima['Outcome']

print(y.value_counts()[0])
print(y.value_counts()[1])

logreg = LogisticRegression(class_weight={1: 500/268})
#logreg = LogisticRegression(class_weight={1: y.value_counts()[0]/y.value_counts()[1]})
# logreg = LogisticRegression()

all_accuracies = cross_val_score(estimator=logreg, X=X, y=y, cv=5, scoring='accuracy')
print(all_accuracies)
print(all_accuracies.mean())

all_f1 = cross_val_score(estimator=logreg, X=X, y=y, cv=5, scoring='f1')
print(all_f1)
print(all_f1.mean())

500
268
[0.7012987  0.66883117 0.68831169 0.71895425 0.66666667]
0.6888124946948476
[0.5        0.38554217 0.38461538 0.4556962  0.4       ]
0.4251707511643458



How to choose Models based on Cross-Validation
We want to have low variance result for CV -> pick a model that has lower variance

If two models have low variance result for CV -> pick a model that has higher mean

Grid Search for Parameter Selection
Machine learning models have hyper-parameters. To know which values of hyper-paramaeters give the best result we need grid search

Question: what does grid search mean?